# Actual Generation per Production Type

Aggregated Generation per Type \[16.1.B&C]

[Entsoe API](https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show#)

Get test data of electricity generation by source (coal, solar, nuclear) for one hour in Czechia. 

User account is needed to download the original csv file from the Entsoe website.

In [1]:
import numpy as np
import pandas as pd

In [2]:
original = pd.read_csv('Actual Generation per Production Type_202101010000-202201010000.csv', na_values=['-', 'n/e'])

In [3]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 23 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Area                                                       8761 non-null   object 
 1   MTU                                                        8761 non-null   object 
 2   Biomass  - Actual Aggregated [MW]                          2390 non-null   float64
 3   Fossil Brown coal/Lignite  - Actual Aggregated [MW]        2390 non-null   float64
 4   Fossil Coal-derived gas  - Actual Aggregated [MW]          2390 non-null   float64
 5   Fossil Gas  - Actual Aggregated [MW]                       2390 non-null   float64
 6   Fossil Hard coal  - Actual Aggregated [MW]                 2390 non-null   float64
 7   Fossil Oil  - Actual Aggregated [MW]                       2390 non-null   float64
 8   Fossil O

In [4]:
original.head(2)

,Area,MTU,Biomass - Actual Aggregated [MW],Fossil Brown coal/Lignite - Actual Aggregated [MW],Fossil Coal-derived gas - Actual Aggregated [MW],Fossil Gas - Actual Aggregated [MW],Fossil Hard coal - Actual Aggregated [MW],Fossil Oil - Actual Aggregated [MW],Fossil Oil shale - Actual Aggregated [MW],Fossil Peat - Actual Aggregated [MW],...,Hydro Run-of-river and poundage - Actual Aggregated [MW],Hydro Water Reservoir - Actual Aggregated [MW],Marine - Actual Aggregated [MW],Nuclear - Actual Aggregated [MW],Other - Actual Aggregated [MW],Other renewable - Actual Aggregated [MW],Solar - Actual Aggregated [MW],Waste - Actual Aggregated [MW],Wind Offshore - Actual Aggregated [MW],Wind Onshore - Actual Aggregated [MW]
0,Czech Republic (CZ),01.01.2021 00:00 - 01.01.2021 01:00 (CET),291.0,3640.0,279.0,312.0,196.0,8.0,NaN,NaN,...,151.0,11.0,NaN,3468.0,51.0,271.0,0.0,23.0,NaN,68.0
1,Czech Republic (CZ),01.01.2021 01:00 - 01.01.2021 02:00 (CET),290.0,3422.0,251.0,314.0,194.0,7.0,NaN,NaN,...,148.0,11.0,NaN,3466.0,52.0,270.0,0.0,23.0,NaN,65.0


In [5]:
# Datetime columns type conversions
from_to = original['MTU'].str.split('-', expand=True)
from_to.columns = ['od', 'do']
from_to['od'] = from_to['od'].str.strip()
from_to['do'] = from_to['do'].str.strip('(CET)').str.strip()
from_to = from_to.apply(pd.to_datetime, dayfirst=True)

In [6]:
df = pd.DataFrame()

In [7]:
# Add to new dataframe
df['od'] = from_to['od']
df['do'] = from_to['do']
df.head()

,od,do
0,2021-01-01 00:00:00,2021-01-01 01:00:00
1,2021-01-01 01:00:00,2021-01-01 02:00:00
2,2021-01-01 02:00:00,2021-01-01 03:00:00
3,2021-01-01 03:00:00,2021-01-01 04:00:00
4,2021-01-01 04:00:00,2021-01-01 05:00:00


In [8]:
df['uhli'] = original.loc[:, 'Fossil Brown coal/Lignite  - Actual Aggregated [MW]': 'Fossil Coal-derived gas  - Actual Aggregated [MW]'].sum(axis=1)
df['uhli'] = df['uhli'] + original['Fossil Hard coal  - Actual Aggregated [MW]']

In [9]:
df['plyn'] = original['Fossil Gas  - Actual Aggregated [MW]']

In [10]:
df['ropa'] = original['Fossil Oil  - Actual Aggregated [MW]']

In [11]:
df['jadro'] = original['Nuclear  - Actual Aggregated [MW]']

In [12]:
df['ostatni'] = original['Other  - Actual Aggregated [MW]']

In [13]:
df['slunce'] = original['Solar  - Actual Aggregated [MW]']

In [14]:
df['vitr'] = original['Wind Onshore  - Actual Aggregated [MW]']

In [15]:
df['voda'] = original['Hydro Run-of-river and poundage  - Actual Aggregated [MW]']

In [16]:
df['biomasa'] = original['Biomass  - Actual Aggregated [MW]']

In [17]:
# df['geotermalni'] = original['Geothermal  - Actual Aggregated [MW]']

In [18]:
df['odpad'] = original['Waste  - Actual Aggregated [MW]']

In [19]:
df['ostatni_oze'] = original['Other renewable  - Actual Aggregated [MW]']

In [20]:
df.to_csv('entsoe-cz-2021.csv', index_label='index')

In [21]:
df.head()

,od,do,uhli,plyn,ropa,jadro,ostatni,slunce,vitr,voda,biomasa,odpad,ostatni_oze
0,2021-01-01 00:00:00,2021-01-01 01:00:00,4115.0,312.0,8.0,3468.0,51.0,0.0,68.0,151.0,291.0,23.0,271.0
1,2021-01-01 01:00:00,2021-01-01 02:00:00,3867.0,314.0,7.0,3466.0,52.0,0.0,65.0,148.0,290.0,23.0,270.0
2,2021-01-01 02:00:00,2021-01-01 03:00:00,3740.0,314.0,8.0,3464.0,52.0,0.0,57.0,147.0,285.0,23.0,271.0
3,2021-01-01 03:00:00,2021-01-01 04:00:00,3713.0,316.0,7.0,3467.0,52.0,0.0,76.0,146.0,291.0,23.0,270.0
4,2021-01-01 04:00:00,2021-01-01 05:00:00,3770.0,316.0,7.0,3471.0,52.0,0.0,70.0,145.0,292.0,24.0,270.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   od           8761 non-null   datetime64[ns]
 1   do           8761 non-null   datetime64[ns]
 2   uhli         2390 non-null   float64       
 3   plyn         2390 non-null   float64       
 4   ropa         2390 non-null   float64       
 5   jadro        2390 non-null   float64       
 6   ostatni      2390 non-null   float64       
 7   slunce       2390 non-null   float64       
 8   vitr         2390 non-null   float64       
 9   voda         2390 non-null   float64       
 10  biomasa      2390 non-null   float64       
 11  odpad        2390 non-null   float64       
 12  ostatni_oze  2390 non-null   float64       
dtypes: datetime64[ns](2), float64(11)
memory usage: 889.9 KB


In [23]:
# One hour in April for Czechia
df.loc[df['od'] == '2021-04-09 13:00:00', 'uhli':]

,uhli,plyn,ropa,jadro,ostatni,slunce,vitr,voda,biomasa,odpad,ostatni_oze
2365,3191.0,895.0,0.0,2828.0,0.0,1448.0,153.0,185.0,290.0,18.0,268.0


In [24]:
# One day in April for Czechia
april_9th = (df['od'] >= '2021-04-09 00:00:00') & (df['od'] < '2021-04-10 00:00:00')
one_day = df.loc[april_9th, 'uhli':].copy()

In [25]:
one_day.head()

,uhli,plyn,ropa,jadro,ostatni,slunce,vitr,voda,biomasa,odpad,ostatni_oze
2352,3406.0,1253.0,0.0,2773.0,0.0,0.0,43.0,191.0,278.0,18.0,266.0
2353,3537.0,1259.0,0.0,2772.0,0.0,0.0,57.0,190.0,278.0,18.0,267.0
2354,3353.0,1255.0,0.0,2757.0,0.0,0.0,71.0,189.0,277.0,17.0,267.0
2355,3327.0,1255.0,0.0,2773.0,0.0,0.0,85.0,190.0,277.0,17.0,264.0
2356,3372.0,1268.0,1.0,2786.0,0.0,0.0,110.0,188.0,275.0,17.0,262.0


In [26]:
columns = ['uhli', 'plyn', 'ropa', 'jadro', 'slunce', 'vitr', 'voda', 'biomasa','odpad', 'ostatni_oze']

In [27]:
one_day = one_day[columns]

In [28]:
one_day.to_csv('one-april-day.csv', index=False)

In [29]:
one_day_pct = one_day.apply(lambda x: np.round(x / np.sum(x) * 100), axis=1)

In [30]:
one_day_pct.head()

,uhli,plyn,ropa,jadro,slunce,vitr,voda,biomasa,odpad,ostatni_oze
2352,41.0,15.0,0.0,34.0,0.0,1.0,2.0,3.0,0.0,3.0
2353,42.0,15.0,0.0,33.0,0.0,1.0,2.0,3.0,0.0,3.0
2354,41.0,15.0,0.0,34.0,0.0,1.0,2.0,3.0,0.0,3.0
2355,41.0,15.0,0.0,34.0,0.0,1.0,2.0,3.0,0.0,3.0
2356,41.0,15.0,0.0,34.0,0.0,1.0,2.0,3.0,0.0,3.0


In [31]:
one_day_pct_int = one_day_pct.apply(pd.to_numeric, downcast='integer')

In [32]:
one_day_pct_int.head()

,uhli,plyn,ropa,jadro,slunce,vitr,voda,biomasa,odpad,ostatni_oze
2352,41,15,0,34,0,1,2,3,0,3
2353,42,15,0,33,0,1,2,3,0,3
2354,41,15,0,34,0,1,2,3,0,3
2355,41,15,0,34,0,1,2,3,0,3
2356,41,15,0,34,0,1,2,3,0,3


In [33]:
one_day_pct_int.to_csv('one-april-day-pct.csv', index=False)

In [34]:
one_day_pct.sum(axis=1)

2352     99.0
2353     99.0
2354     99.0
2355     99.0
2356     99.0
2357     99.0
2358    100.0
2359    101.0
2360    100.0
2361    100.0
2362    100.0
2363     99.0
2364    100.0
2365    100.0
2366    100.0
2367    100.0
2368     99.0
2369     98.0
2370     99.0
2371     99.0
2372     99.0
2373     98.0
2374     99.0
2375     99.0
dtype: float64